Test your code with this notebook; replace your own code blocks in the for loop
Your code should work if 

In [6]:
def RGB_func(X): 
    '''
    This is the 'fake' function to genration RGB from the x_A, x_B, x_C
    Please replace this function with the actual experimental data
    x is normalized. 
    '''
    X = np.array(X)
    RGB = X.dot(np.array([[1,0.1,0.0],[0.0,0.8,0.1],[0.3,0.0,1]]))
    return np.around(RGB*255+0.02* np.random.randn(*RGB.shape)*255)

In [7]:
import numpy as np

flag = False
############### TASK 1: Initial Conditions ###################
############### Please Change with your own code #############
X_init = [[10,100,90],[50, 20, 130]] 

###############################################################

##******************Check the X_init format****************#
for x in np.array(X_init):
    if len(x) != 3:
        print('ERROR! each row needs to be 3 elements for color A, B and C')
        print('you have', len(x), ' elements in one of the rows')
        flag = True
    if abs(sum(x)-200)> 0.01*200:
        print('ERROR! each row needs to sum up closed to 200 (or 198 - 200) [uL]')
        print('you have a sum of', sum(x), ' [uL] in one of the rows')
        flag = True
##*********************************************************#

if flag != True:
    X_new = np.array(X_init)
    X = []

    num_rounds = 25
    for i in range(num_rounds):
        if X ==[]:
            X = X_new/200
        else:
            X = np.concatenate([X, X_new/200])
            if len(X)>25:
                X = X[0:25]

        RGB_ref = [np.array([ 83., 108.,  76.])+0.01* np.random.randn(3)*255] # each value is in the range 0-255
        RGB_exp = np.array(RGB_func(X))
        

#######TASK 2: Optimization Metric (e.g. error function)############
############### Please Change with your own code #############

        def MSE_func(RGB_exp,RGB_ref):
            from sklearn.metrics import mean_squared_error
            '''
            Caclulate the metric for optimization: 
            Mean Square Error of experimental RGB measurements and the targeted RGB
            '''
            Y = []
            for i in range(len(RGB_exp)):
                Y.append([mean_squared_error(RGB_exp[i],RGB_ref[0])])
            return np.array(Y)
        
        Y = MSE_func(RGB_exp, RGB_ref)
####################################################################

##******************Check the Y format****************##
        if len(np.shape(Y))==2:
            if np.shape(Y)[1] != 1:
                print('we need a two dimensional array with 1 element at each row')
                print(np.shape(Y)[1])
                flag == True
        else:
            print('we need a two dimensional array with 1 element at each row')
            flag == True
        if flag == True:
            break 
##*********************************************************#



#######TASK 3: Optimization Metric (e.g. error function)############
############### Please Change with your own code ##################

        def optimizer_func(X, Y, BatchSize):
            '''
            Bayesian Optimizer Function 

            BatchSize is the number of suggestions for the next rounds
            X should be the input variables
            Y should be the metric to be optimized
            '''
            import GPy
            from GPyOpt.methods import BayesianOptimization

            bds = [{'name': 'x1', 'type': 'continuous', 'domain': (0, 1)},
                   {'name': 'x2', 'type': 'continuous', 'domain': (0, 1)},
                   {'name': 'x3', 'type': 'continuous', 'domain': (0, 1)},
                  ]


            constraints = [{'name': 'constr_1', 
                            'constraint': 'x[:,0] + x[:,1] + x[:,2] -(1 + 0.005)'},###<= 0
                           {'name': 'constr_2', 
                            'constraint': '(1- 0.005) - (x[:,0] + x[:,1] + x[:,2]) '}]###<= 0

            kernel = GPy.kern.Matern52(input_dim=len(bds), ARD = True)

            optimizer = BayesianOptimization(f=None, 
                                             domain=bds,
                                             constraints = constraints,
                                             model_type='GP',
                                             acquisition_type ='EI',
                                             acquisition_jitter = 0.1,
                                             X=X,
                                             Y=Y,
                                             evaluator_type = 'local_penalization',
                                             batch_size = BatchSize,
                                             normalize_Y= True,
                                             #noise_var = 0.02**2,
                                             kernel = kernel
                                             )
            return optimizer


        batch_size = 2
        opt = optimizer_func(np.array(X), np.array(Y), batch_size)
        X_new = opt.suggest_next_locations()*200
####################################################################


##******************Check the X_new format****************##
        for x in np.array(X_init):
            if len(x) != 3:
                print('ERROR! each row needs to be 3 elements for color A, B and C')
                print('you have', len(x), ' elements in one of the rows')
                flag = True
            if abs(sum(x)-200)> 0.01*200:
                print('ERROR! each row needs to sum up closed to 200 (or 198 - 200) [uL]')
                print('you have a sum of', sum(x), ' [uL] in one of the rows')
                flag = True
        if flag == True:
            break
##*********************************************************#


            
##############TASK 4: Early Stop Condition#########################
############### You can change it or leave it like this #############

        if len(RGB_exp)>25:
            print('Reach the max number of samples')
            break
    
        def color_difference(rgb, rgb_ref):
            return abs(rgb_ref[0]-rgb[0]), abs(rgb_ref[1]-rgb[1]), abs(rgb_ref[2]-rgb[2])
        diff = []
        for rgb in RGB_exp:
            rgb_ref = RGB_ref[0]
            r_err, g_err, b_err = color_difference(rgb, rgb_ref)
            diff.append((r_err<=8) & (g_err<=8) & (b_err<=8))
        if np.sum(diff)>=1:
            print("Success! Early stop after collecting", len(diff), "samples")
            print("Good match at ", np.arange(len(diff))[np.array(diff) == 1])
            break
####################################################################

print("Lowest Error:", np.round(np.min(np.sqrt(Y))))
print("Lowest Error RGB", RGB_exp[np.argmin(np.sqrt(Y))])
print("Lowest ref RGB", np.around(RGB_ref[0]))

Success! Early stop after collecting 14 samples
Good match at  [13]
Lowest Error: 1.0
Lowest Error RGB [ 82. 109.  79.]
Lowest ref RGB [ 81. 111.  78.]
